In [ ]:
# ==================================================================================================
# Yufei Li
# Princeton University
# yl5385@princeton.edu

# Feburay 2023

# Note:In this demo, the neural network is synthesized using the TensorFlow (verion: 2.11.0) framework. 
# Please install TensorFlow according to the official guidance, then import TensorFlow and other dependent modules.
# ==================================================================================================

!pip install pandas numpy matplotlib
!pip install tensorflow
!pip install optuna

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import copy
import csv
import math
import cmath
import time

import optuna

inFilename = "Input_Yo20op272_ana.csv"
outFilename = "Output_Yo20op272_ana.csv"

Input = pd.read_csv(inFilename,header=None)
Output = pd.read_csv(outFilename,header=None)

print(Input.dtypes)
print(Output.dtypes)

inputs = []
outputs = []

inputs = np.array(Input)
outputs = np.array(Output)
print(inputs)
print(outputs)

In [ ]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation

num_inputs = len(inputs)
print("Total Number of Dataset is:",num_inputs)
randomize = np.arange(num_inputs)
print(randomize)

inputs_origin = copy.deepcopy(inputs[randomize])
outputs_origin = copy.deepcopy(outputs[randomize])
print(inputs_origin)

random.Random(5).shuffle(randomize)
print(randomize)
# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs_real = copy.deepcopy(inputs_origin[randomize])
outputs_real = copy.deepcopy(outputs_origin[randomize])
print(inputs_real)
print(outputs_real)

# Split the recordings (group of samples) into two sets: training and testing
TRAIN_SPLIT = int(0.7 * num_inputs)
inputs_train, inputs_test = np.split(inputs_real, [TRAIN_SPLIT])
outputs_train, outputs_test = np.split(outputs_real, [TRAIN_SPLIT])

num_inputs_train = len(inputs_train)
print("Total Number of training Dataset is:",num_inputs_train)
print("Dataset randomization and separation complete!")

In [ ]:
EPOCHS = 500

def objective(trial):
    #Clear clutter from previous tf.keras session graphs.
    #tf.keras.backend.clear_session()
    
    #Define normalization layer
    Normlayer1=tf.keras.layers.Normalization()
    Normlayer1.adapt(inputs_train)
    
    #Model construction (2140 dataset)
    #Optimize the numbers of layers and their units.
    model = tf.keras.Sequential()
    model.add(Normlayer1)    
    model.add(tf.keras.layers.Dense(43, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(56, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(43, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(8))

    #Defining learning rate schedule
    LR_INIT = trial.suggest_float("lr_init", 1e-3, 5e-1, log=True)
    DECAY_EPOCHS = trial.suggest_int("decay_epochs", 10, 200, log=True)
    DECAY_RATE = trial.suggest_float("decay_rate", 0.4, 0.9, log=True)
    BATCH_SIZE = trial.suggest_categorical("batch_size", [16, 32, 64, 128, 256, 512, 1024])
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    LR_INIT,
    decay_steps=math.ceil(num_inputs_train/BATCH_SIZE)*DECAY_EPOCHS,
    decay_rate=DECAY_RATE,
    staircase=True)
    
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=opt, loss='mse', metrics=['mse'])
    
    history = model.fit(inputs_train, outputs_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(inputs_test, outputs_test), verbose=0)

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(inputs_test, outputs_test, verbose=0)
    return score[1]
print("Optimization object construction complete!")

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200)
    
    print("Number of finished trials: ", len(study.trials))       
    
    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
optuna.visualization.plot_optimization_history(study)#Plotting the optimization history of the study.

In [ ]:
optuna.visualization.plot_intermediate_values(study)#Visualizing the Learning Curves of the Trials

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)#Visualizing High-dimensional Parameter Relationships

In [ ]:
optuna.visualization.plot_parallel_coordinate(study, params=['lr_init', 'decay_epochs', 'decay_rate'])#Selecting Parameters to Visualize

In [ ]:
optuna.visualization.plot_contour(study)#Visualizing Parameter Relationships

In [ ]:
optuna.visualization.plot_contour(study, params=['n_units_l0', 'n_units_l1'])#Selecting Parameters to Visualize

In [ ]:
optuna.visualization.plot_slice(study)#Plotting the accuracies for each hyperparameter for each trial. Visualizing Individual Parameters

In [ ]:
optuna.visualization.plot_slice(study, params=['n_units_l0', 'n_units_l1'])#Selecting Parameters to Visualize

In [ ]:
optuna.visualization.plot_param_importances(study)#Visualizing Parameter Importances

In [ ]:
optuna.visualization.plot_edf(study)